**Details about the concatenation process can be found *[here](https://docs.google.com/document/d/1n1Gm-SF5vY-YdLTMA5cNQVpJgr_d7OHwxp7qz6P6Do0/edit?usp=sharing).***

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount = True)

Mounted at /content/gdrive


In [ ]:
#import libraries
import pandas as pd
import numpy as np

In [ ]:
aid = pd.read_csv('/content/gdrive/My Drive/[Intro Data] Group Project (shared)/Data/aid_received_total_us_inflation_adjusted.csv')
broadband = pd.read_csv('/content/gdrive/My Drive/[Intro Data] Group Project (shared)/Data/broadband_subscribers.csv')
democracy1 = pd.read_csv('/content/gdrive/My Drive/[Intro Data] Group Project (shared)/Data/democracy_score_use_as_color.csv')
democracy2 = pd.read_csv('/content/gdrive/My Drive/[Intro Data] Group Project (shared)/Data/dirdemo_idea.csv')
line_phone = pd.read_csv('/content/gdrive/My Drive/[Intro Data] Group Project (shared)/Data/fixed_line_subscribers_per_100_people.csv')
fdi = pd.read_csv('/content/gdrive/My Drive/[Intro Data] Group Project (shared)/Data/foreign_direct_investment_net_inflows_percent_of_gdp.csv')
inequality = pd.read_csv('/content/gdrive/My Drive/[Intro Data] Group Project (shared)/Data/gini.csv')
income = pd.read_csv('/content/gdrive/My Drive/[Intro Data] Group Project (shared)/Data/gdp_pcap.csv')
#trade = pd.read_csv('/content/gdrive/My Drive/[Intro Data] Group Project (shared)/Data/broadband_subscribers.csv') -> not going to use this for total trade
gdp = pd.read_csv('/content/gdrive/My Drive/[Intro Data] Group Project (shared)/Data/total_gdp_us_inflation_adjusted.csv')
import_vol = pd.read_csv('/content/gdrive/My Drive/[Intro Data] Group Project (shared)/Data/imports_percent_of_gdp.csv')
export_vol = pd.read_csv('/content/gdrive/My Drive/[Intro Data] Group Project (shared)/Data/exports_percent_of_gdp.csv')
internet = pd.read_csv('/content/gdrive/My Drive/[Intro Data] Group Project (shared)/Data/internet_users.csv')

In [ ]:
list_df = [democracy2 , aid, line_phone, internet, fdi, inequality, income, gdp, import_vol, export_vol] #no broadband + democracy 1
country = democracy2.country

In [ ]:
for i in range(len(list_df)):
  list_df[i] = list_df[i].set_index('country').loc[:, '1989' : '2019'].reset_index()
  null_counts = list_df[i].isnull().sum(axis=1)
  mask = null_counts <= list_df[i].shape[1]/5
  list_df[i].fillna(method = 'ffill', inplace = True)
  list_df[i] = list_df[i].loc[mask]
  country = country[country.isin(list_df[i].country)]

In [ ]:
for i in range(len(list_df)):
  list_df[i] = list_df[i][list_df[i].country.isin(country)]

In [ ]:
for i in range(len(list_df)):
  print(list_df[i].shape)

(90, 32)
(90, 32)
(90, 32)
(90, 32)
(90, 32)
(90, 32)
(90, 32)
(90, 32)
(90, 32)
(90, 32)


In [ ]:
def str_to_float(a):
    #if x and x[0] == '-':
    if type(a) == str:
      x = a.replace('\u2212', '-').replace('\u03BC','')
      if x.isspace():
        x = float('nan')
      elif x.startswith('-'):
        if x.endswith('M'):
          return -1 * float(x[1:-1]) * 1e6
        elif x.endswith('K'):
          return -1 * float(x[1:-1]) * 1e3
        elif x.endswith('B'):
          return -1 * float(x[1:-1]) * 1e9
        elif x.endswith('k'):
          return -1 * float(x[1:-1]) * 1e3
        elif x.endswith('µ'):
          return -1 * float(x[1:-1])
        elif x.endswith('TR'):
          return -1 * float(x[1:-2]) *1e12
        else:
          return -1 * float(x[1:])
      else:
        if x.endswith('M'):
          return float(x[:-1]) * 1e6
        elif x.endswith('K'):
          return float(x[:-1]) * 1e3
        elif x.endswith('B'):
          return float(x[:-1]) * 1e9
        elif x.endswith('k'):
          return float(x[:-1]) * 1e3
        elif x.endswith('TR'):
          return float(x[:-2]) * 1e12
        elif x.endswith('µ'):
          return float(x[:-1])
        else:
          return float(x[:])


In [ ]:
melted_df_base = list_df[0].melt(id_vars = 'country', var_name = 'Year', value_name= '{}'.format(0))
df_base = melted_df_base.sort_values(['country', 'Year'])
df_base

,country,Year,0
0,Albania,1989,0.000
90,Albania,1990,0.000
180,Albania,1991,0.764
270,Albania,1992,0.922
360,Albania,1993,0.915
...,...,...,...
2429,Zimbabwe,2015,11.600
2519,Zimbabwe,2016,11.500
2609,Zimbabwe,2017,11.400
2699,Zimbabwe,2018,9.080


In [ ]:
for i in range(1, len(list_df)):
    melted_df = list_df[i].melt(id_vars = 'country', var_name = 'Year', value_name= '{}'.format(i))
    df = melted_df.sort_values(['country', 'Year'])
    if type(df.iloc[1,2]) == type('1'):
      df_base = pd.concat([df_base, df.iloc[:,2].apply(str_to_float)], axis = 1)
    else:
      df_base = pd.concat([df_base, df.iloc[:,2]], axis = 1)

In [ ]:
df_base.columns = ['Country' , 'Year' , 'Democracy Score', 'Aid Received', 'Line Phone', 'Internet', 'FDI' , 'Inequality', 'Income' , 'GDP' , 'Import' , 'Export']

In [ ]:
df_base.Year.unique()

array(['1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996',
       '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
       '2013', '2014', '2015', '2016', '2017', '2018', '2019'],
      dtype=object)

In [ ]:
df_base['Democracy Growth'] = ''
for i in df_base.Country.unique():
  x = df_base.loc[df_base.Country == i, 'Democracy Score'].copy()
  df_base.loc[df_base.Country == i, 'Democracy Growth'] = -1*x.diff(-1)

In [ ]:
df_base = df_base.drop(df_base[df_base.Year == '1989'].index)

In [ ]:
x = df_base[df_base.Internet.isnull()]
x

,Country,Year,Democracy Score,Aid Received,Line Phone,Internet,FDI,Inequality,Income,GDP,Import,Export,Democracy Growth


In [ ]:
df_base.Internet.loc[(df_base.Country == 'Thailand') & (df_base.Year == '1991')] = float(50)

<ipython-input-41-17513aaf53e1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_base.Internet.loc[(df_base.Country == 'Thailand') & (df_base.Year == '1991')] = float(50)


In [ ]:
df_base.dropna(inplace = True)
df_base.insert(3, 'Democracy Growth', df_base.pop('Democracy Growth'))
df_base.reset_index(drop = True, inplace = True)

In [ ]:
df_base

,Country,Year,Democracy Score,Democracy Growth,Aid Received,Line Phone,Internet,FDI,Inequality,Income,GDP,Import,Export
0,Albania,1990,0.000,0.764,16300000.0,1.22,0.0,0.00,26.9,5020.0,5.280000e+09,24.0,15.40
1,Albania,1991,0.764,0.158,463000000.0,1.27,0,0.00,27.0,3650.0,3.800000e+09,28.6,7.48
2,Albania,1992,0.922,-0.007,564000000.0,1.38,0.0,3.07,27.0,3430.0,3.530000e+09,96.3,12.50
3,Albania,1993,0.915,7.085,404000000.0,1.35,0.0,4.89,27.0,3770.0,3.860000e+09,64.5,16.00
4,Albania,1994,8.000,-7.077,223000000.0,1.32,0.0,2.82,27.0,4090.0,4.190000e+09,41.1,12.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2605,Zimbabwe,2014,11.600,-0.0,718000000.0,2.42,16.4,2.43,43.7,2320.0,1.960000e+10,33.7,20.90
2606,Zimbabwe,2015,11.600,-0.1,818000000.0,2.42,22.7,2.00,43.9,2310.0,2.000000e+10,37.6,19.20
2607,Zimbabwe,2016,11.500,-0.1,688000000.0,2.18,23.1,1.67,44.1,2290.0,2.010000e+10,31.3,19.90
2608,Zimbabwe,2017,11.400,-2.32,754000000.0,1.86,24.4,1.75,44.3,2330.0,2.110000e+10,30.4,19.70


In [ ]:
df_base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2610 entries, 0 to 2609
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Country           2610 non-null   object 
 1   Year              2610 non-null   object 
 2   Democracy Score   2610 non-null   float64
 3   Democracy Growth  2610 non-null   object 
 4   Aid Received      2610 non-null   float64
 5   Line Phone        2610 non-null   float64
 6   Internet          2610 non-null   object 
 7   FDI               2610 non-null   float64
 8   Inequality        2610 non-null   float64
 9   Income            2610 non-null   float64
 10  GDP               2610 non-null   float64
 11  Import            2610 non-null   float64
 12  Export            2610 non-null   float64
dtypes: float64(9), object(4)
memory usage: 265.2+ KB


In [ ]:
df_base.drop('Democracy Growth', axis = 1, inplace = True)

In [ ]:
from google.colab import files
df_base.to_csv('df_base.csv', index = False)
files.download('df_base.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>